### Running a Logistic Regression Model on Multiple Features
----

In [107]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Oct  5 11:00:56 2018

@author: RaviPandey
"""


# =============================================================================
# Importing Libraries
# =============================================================================

import numpy as np
import pandas as pd


# =============================================================================
# Loading and Exploring data
# =============================================================================

df = pd.read_csv('../data/loan.csv')

df = df.loc[ 0:10000, ['loan_amnt', 'int_rate', 'annual_inc', 'inq_last_6mths', 
                   'open_acc', 'revol_bal', 'revol_util', 'total_acc', 'dti',
                   'term', 'grade', 'home_ownership', 'verification_status',
                        'purpose', 'delinq_2yrs', 'loan_status']]



/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (19,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [108]:
# dimension of the dataframe
df.shape


(10001, 16)

In [109]:

# Exploring column names
df.columns



Index(['loan_amnt', 'int_rate', 'annual_inc', 'inq_last_6mths', 'open_acc',
       'revol_bal', 'revol_util', 'total_acc', 'dti', 'term', 'grade',
       'home_ownership', 'verification_status', 'purpose', 'delinq_2yrs',
       'loan_status'],
      dtype='object')

In [110]:
# Exploring first few records
df.head()



,loan_amnt,int_rate,annual_inc,inq_last_6mths,open_acc,revol_bal,revol_util,total_acc,dti,term,grade,home_ownership,verification_status,purpose,delinq_2yrs,loan_status
0,5000.0,10.65,24000.0,1.0,3.0,13648.0,83.7,9.0,27.65,36 months,B,RENT,Verified,credit_card,0.0,Fully Paid
1,2500.0,15.27,30000.0,5.0,3.0,1687.0,9.4,4.0,1.00,60 months,C,RENT,Source Verified,car,0.0,Charged Off
2,2400.0,15.96,12252.0,2.0,2.0,2956.0,98.5,10.0,8.72,36 months,C,RENT,Not Verified,small_business,0.0,Fully Paid
3,10000.0,13.49,49200.0,1.0,10.0,5598.0,21.0,37.0,20.00,36 months,C,RENT,Source Verified,other,0.0,Fully Paid
4,3000.0,12.69,80000.0,0.0,15.0,27783.0,53.9,38.0,17.94,60 months,B,RENT,Source Verified,other,0.0,Current


In [111]:
# Getting summarized information
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   loan_amnt            10001 non-null  float64
 1   int_rate             10001 non-null  float64
 2   annual_inc           10001 non-null  float64
 3   inq_last_6mths       10001 non-null  float64
 4   open_acc             10001 non-null  float64
 5   revol_bal            10001 non-null  float64
 6   revol_util           9998 non-null   float64
 7   total_acc            10001 non-null  float64
 8   dti                  10001 non-null  float64
 9   term                 10001 non-null  object 
 10  grade                10001 non-null  object 
 11  home_ownership       10001 non-null  object 
 12  verification_status  10001 non-null  object 
 13  purpose              10001 non-null  object 
 14  delinq_2yrs          10001 non-null  float64
 15  loan_status          10001 non-null 

In [112]:

numeric_features = ['loan_amnt', 'int_rate', 'annual_inc', 'inq_last_6mths', 
                   'open_acc', 'revol_bal', 'revol_util', 'total_acc', 'dti']

categorical_features = ['term', 'grade', 'home_ownership', 'verification_status',
                        'purpose', 'delinq_2yrs']



In [113]:
# =============================================================================
# Applying Transformation to numeric variable
# =============================================================================


# Imputer - missing Value
# StandardScaler
# MaxAbsScaler
# MinMaxScaler
# QuantileTransformers - Uniform Distribution
# PowerTransformer - Gaussian Distrbution
# Normalizer
# FunctionTransformer - Custom Transformer

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer


In [114]:

numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
        ('sqrt_transformer', FunctionTransformer(np.sqrt))])
  


In [115]:

# =============================================================================
# Applying transformation to Categorical Variable
# =============================================================================


from sklearn.preprocessing import OneHotEncoder
   
categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
        ('onehot' , OneHotEncoder(sparse=False))])


In [116]:
# =============================================================================
# Creating Preprocessor
# =============================================================================

from sklearn.compose import ColumnTransformer
    
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


In [117]:
# =============================================================================
# Model
# =============================================================================

from sklearn.linear_model import LogisticRegression
    
model = Pipeline(steps=[
        ('preprocess', preprocessor),
        ('lr_model', LogisticRegression())])


In [118]:

# =============================================================================
# Applying Model
# =============================================================================
X = df.drop('loan_status', axis=1)
X = X.iloc[1:1000, :]
y = np.where(df['loan_status'].isin(['Charged Off', 'Default']), 1, 0)
y = y[1:1000]

def file_modify(dataset):
    print("Start")
    dataset['home_ownership'] = np.where(
            dataset.home_ownership.isin(['NONE', 'ANY']), 'MORTGAGE', 
            dataset.home_ownership)
    dataset['purpose'] = np.where(
            dataset.purpose.isin(['debt_consolidation', 'credit_card', 'home_improvement']), 
            dataset.purpose, 
            'other')
    dataset['delinq_2yrs'] = np.where(
            dataset.delinq_2yrs > 0, 1, 0 )
    return dataset


In [119]:

X = file_modify(X)

Start


In [120]:
model.fit(X, y)


/home/analyst/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('sqrt_transformer',
                                                                   FunctionTransformer(func=<ufunc 'sqrt'>))]),
                                                  ['loan_amnt', 'int_rate',
                                                   'annual_inc',
                                                   'inq_last_6mths', 'open_acc',
                                                   'revol_bal', 'revol_util',
                                                   'total_acc', 'dti']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                         

In [121]:
pred = model.predict(X)
pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [122]:
# =============================================================================
# Dumping Model
# =============================================================================
import joblib

joblib.dump(model, 'first_go.joblib')

['first_go.joblib']

## Feature Extraction
--- 
Select the K-Best Features

In [123]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Oct  5 13:40:11 2018

@author: RaviPandey
"""

# =============================================================================
# Importing Libraries
# =============================================================================

import numpy as np
import pandas as pd

# =============================================================================
# Loading and Exploring data
# =============================================================================

df = pd.read_csv('../data/loan.csv')

df = df.loc[ 0:9999, ['loan_amnt', 'int_rate', 'annual_inc', 'inq_last_6mths', 
                   'open_acc', 'revol_bal', 'revol_util', 'total_acc', 'dti',
                   'term', 'grade', 'home_ownership', 'verification_status',
                        'purpose', 'delinq_2yrs', 'loan_status']]

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (19,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [124]:
# dimension of the dataframe
df.shape

(10000, 16)

In [125]:
# Exploring column names
df.columns

Index(['loan_amnt', 'int_rate', 'annual_inc', 'inq_last_6mths', 'open_acc',
       'revol_bal', 'revol_util', 'total_acc', 'dti', 'term', 'grade',
       'home_ownership', 'verification_status', 'purpose', 'delinq_2yrs',
       'loan_status'],
      dtype='object')

In [126]:
# Exploring first few records
df.head()


,loan_amnt,int_rate,annual_inc,inq_last_6mths,open_acc,revol_bal,revol_util,total_acc,dti,term,grade,home_ownership,verification_status,purpose,delinq_2yrs,loan_status
0,5000.0,10.65,24000.0,1.0,3.0,13648.0,83.7,9.0,27.65,36 months,B,RENT,Verified,credit_card,0.0,Fully Paid
1,2500.0,15.27,30000.0,5.0,3.0,1687.0,9.4,4.0,1.00,60 months,C,RENT,Source Verified,car,0.0,Charged Off
2,2400.0,15.96,12252.0,2.0,2.0,2956.0,98.5,10.0,8.72,36 months,C,RENT,Not Verified,small_business,0.0,Fully Paid
3,10000.0,13.49,49200.0,1.0,10.0,5598.0,21.0,37.0,20.00,36 months,C,RENT,Source Verified,other,0.0,Fully Paid
4,3000.0,12.69,80000.0,0.0,15.0,27783.0,53.9,38.0,17.94,60 months,B,RENT,Source Verified,other,0.0,Current


In [127]:
# Getting summarized information
df.info()

numeric_features = ['loan_amnt', 'int_rate', 'annual_inc', 'inq_last_6mths', 
                   'open_acc', 'revol_bal', 'revol_util', 'total_acc', 'dti']

categorical_features = ['term', 'grade', 'home_ownership', 'verification_status',
                        'purpose', 'delinq_2yrs']


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   loan_amnt            10000 non-null  float64
 1   int_rate             10000 non-null  float64
 2   annual_inc           10000 non-null  float64
 3   inq_last_6mths       10000 non-null  float64
 4   open_acc             10000 non-null  float64
 5   revol_bal            10000 non-null  float64
 6   revol_util           9997 non-null   float64
 7   total_acc            10000 non-null  float64
 8   dti                  10000 non-null  float64
 9   term                 10000 non-null  object 
 10  grade                10000 non-null  object 
 11  home_ownership       10000 non-null  object 
 12  verification_status  10000 non-null  object 
 13  purpose              10000 non-null  object 
 14  delinq_2yrs          10000 non-null  float64
 15  loan_status          10000 non-null  

In [128]:
# =============================================================================
# Applying Transformation to numeric variable
# =============================================================================


# Imputer - missing Value
# StandardScaler
# MaxAbsScaler
# MinMaxScaler
# QuantileTransformers - Uniform Distribution
# PowerTransformer - Gaussian Distrbution
# Normalizer
# FunctionTransformer - Custom Transformer

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer


numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
        ('log_transformer', FunctionTransformer(np.sqrt))])


In [129]:
# =============================================================================
# Applying transformation to Categorical Variable
# =============================================================================


from sklearn.preprocessing import OneHotEncoder
   
categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
        ('onehot' , OneHotEncoder(sparse=False))])


In [130]:
# =============================================================================
# Creating Preprocessor
# =============================================================================

from sklearn.compose import ColumnTransformer
    
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


In [131]:
# =============================================================================
# Feature Selection - Select K best Features
# =============================================================================

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# f_regression, mutual_info_regression
# For classification: chi2, f_classif, mutual_info_classif


In [132]:
# =============================================================================
# Model with feature selection
# =============================================================================

from sklearn.linear_model import LogisticRegression
    
model = Pipeline(steps=[
        ('preprocess', preprocessor),
        ('feature_selection', SelectKBest(score_func= chi2, k = 5) ),
        ('lr_model', LogisticRegression())])


In [133]:
# =============================================================================
# Applying Model
# =============================================================================
X = df.drop('loan_status', axis=1)
X = X.iloc[1:1000, :]
y = np.where(df['loan_status'].isin(['Charged Off', 'Default']), 1, 0)
y = y[1:1000]


In [134]:
def file_modify(dataset):
    print("Start")
    dataset['home_ownership'] = np.where(
            dataset.home_ownership.isin(['NONE', 'ANY']), 'MORTGAGE', 
            dataset.home_ownership)
    dataset['purpose'] = np.where(
            dataset.purpose.isin(['debt_consolidation', 'credit_card', 'home_improvement']), 
            dataset.purpose, 
            'other')
    dataset['delinq_2yrs'] = np.where(
            dataset.delinq_2yrs > 0, 1, 0 )
    return dataset


In [135]:
X = file_modify(X)

model.fit(X, y)

pred = model.predict(X)


Start


In [136]:
# =============================================================================
# Dumping Model
# =============================================================================

import joblib

joblib.dump(model, 'model_with_feature_selection_1.joblib')

['model_with_feature_selection_1.joblib']

### Feature Extraction: Select From Models
----

In [137]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Oct  5 13:40:11 2018

@author: RaviPandey
"""

# =============================================================================
# Importing Libraries
# =============================================================================

import numpy as np
import pandas as pd


# =============================================================================
# Loading and Exploring data
# =============================================================================

df = pd.read_csv('../data/loan.csv')

df = df.loc[ 0:10000, ['loan_amnt', 'int_rate', 'annual_inc', 'inq_last_6mths', 
                   'open_acc', 'revol_bal', 'revol_util', 'total_acc', 'dti',
                   'term', 'grade', 'home_ownership', 'verification_status',
                        'purpose', 'delinq_2yrs', 'loan_status']]


/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (19,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [138]:
# dimension of the dataframe
df.shape

# Exploring column names
df.columns

# Exploring first few records
df.head()

# Getting summarized information
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   loan_amnt            10001 non-null  float64
 1   int_rate             10001 non-null  float64
 2   annual_inc           10001 non-null  float64
 3   inq_last_6mths       10001 non-null  float64
 4   open_acc             10001 non-null  float64
 5   revol_bal            10001 non-null  float64
 6   revol_util           9998 non-null   float64
 7   total_acc            10001 non-null  float64
 8   dti                  10001 non-null  float64
 9   term                 10001 non-null  object 
 10  grade                10001 non-null  object 
 11  home_ownership       10001 non-null  object 
 12  verification_status  10001 non-null  object 
 13  purpose              10001 non-null  object 
 14  delinq_2yrs          10001 non-null  float64
 15  loan_status          10001 non-null 

In [139]:


numeric_features = ['loan_amnt', 'int_rate', 'annual_inc', 'inq_last_6mths', 
                   'open_acc', 'revol_bal', 'revol_util', 'total_acc', 'dti']

categorical_features = ['term', 'grade', 'home_ownership', 'verification_status',
                        'purpose', 'delinq_2yrs']








In [140]:
# =============================================================================
# Applying Transformation to numeric variable
# =============================================================================


# Imputer - missing Value
# StandardScaler
# MaxAbsScaler
# MinMaxScaler
# QuantileTransformers - Uniform Distribution
# PowerTransformer - Gaussian Distrbution
# Normalizer
# FunctionTransformer - Custom Transformer

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer


numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
        ('log_transformer', FunctionTransformer(np.sqrt))])


In [141]:
# =============================================================================
# Applying transformation to Categorical Variable
# =============================================================================


from sklearn.preprocessing import OneHotEncoder
   
categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
        ('onehot' , OneHotEncoder(sparse=False))])


In [142]:
# =============================================================================
# Creating Preprocessor
# =============================================================================

from sklearn.compose import ColumnTransformer
    
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


In [143]:
# =============================================================================
# Feature Selection - Select from models
# =============================================================================

from sklearn import linear_model
reg = linear_model.Lasso(alpha = 0.1)

from sklearn.feature_selection import SelectFromModel


In [144]:
# =============================================================================
# Model with feature selection
# =============================================================================

from sklearn.linear_model import LogisticRegression
    
model = Pipeline(steps=[
        ('preprocess', preprocessor),
        ('feature_selection', SelectFromModel(reg) ),
        ('lr_model', LogisticRegression())])


In [145]:
# =============================================================================
# Applying Model
# =============================================================================



X = df.drop('loan_status', axis=1)
X = X.iloc[1:1000, :]
y = np.where(df['loan_status'].isin(['Charged Off', 'Default']), 1, 0)
y = y[1:1000]


In [146]:
def file_modify(dataset):
    print("Start")
    dataset['home_ownership'] = np.where(
            dataset.home_ownership.isin(['NONE', 'ANY']), 'MORTGAGE', 
            dataset.home_ownership)
    dataset['purpose'] = np.where(
            dataset.purpose.isin(['debt_consolidation', 'credit_card', 'home_improvement']), 
            dataset.purpose, 
            'other')
    dataset['delinq_2yrs'] = np.where(
            dataset.delinq_2yrs > 0, 1, 0 )
    return dataset


In [147]:
X = file_modify(X)

model.fit(X, y)

pred = model.predict(X)


Start


In [148]:
# =============================================================================
# Dumping Model
# =============================================================================


import joblib

joblib.dump(model, 'model_with_feature_selection_2.joblib')


['model_with_feature_selection_2.joblib']

### Adding Another Feature Extraction
----
* PCA - Dimensionality Reduction

In [149]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Oct  5 13:40:11 2018

@author: RaviPandey
"""

# =============================================================================
# Importing Libraries
# =============================================================================

import numpy as np
import pandas as pd

In [150]:
# =============================================================================
# Loading and Exploring data
# =============================================================================

df = pd.read_csv('../data/loan.csv')

df = df.loc[ 0:9999, ['loan_amnt', 'int_rate', 'annual_inc', 'inq_last_6mths', 
                   'open_acc', 'revol_bal', 'revol_util', 'total_acc', 'dti',
                   'term', 'grade', 'home_ownership', 'verification_status',
                        'purpose', 'delinq_2yrs', 'loan_status']]


/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (19,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [151]:
# dimension of the dataframe
df.shape

(10000, 16)

In [152]:
# Exploring column names
df.columns


Index(['loan_amnt', 'int_rate', 'annual_inc', 'inq_last_6mths', 'open_acc',
       'revol_bal', 'revol_util', 'total_acc', 'dti', 'term', 'grade',
       'home_ownership', 'verification_status', 'purpose', 'delinq_2yrs',
       'loan_status'],
      dtype='object')

In [153]:
# Exploring first few records
df.head()


,loan_amnt,int_rate,annual_inc,inq_last_6mths,open_acc,revol_bal,revol_util,total_acc,dti,term,grade,home_ownership,verification_status,purpose,delinq_2yrs,loan_status
0,5000.0,10.65,24000.0,1.0,3.0,13648.0,83.7,9.0,27.65,36 months,B,RENT,Verified,credit_card,0.0,Fully Paid
1,2500.0,15.27,30000.0,5.0,3.0,1687.0,9.4,4.0,1.00,60 months,C,RENT,Source Verified,car,0.0,Charged Off
2,2400.0,15.96,12252.0,2.0,2.0,2956.0,98.5,10.0,8.72,36 months,C,RENT,Not Verified,small_business,0.0,Fully Paid
3,10000.0,13.49,49200.0,1.0,10.0,5598.0,21.0,37.0,20.00,36 months,C,RENT,Source Verified,other,0.0,Fully Paid
4,3000.0,12.69,80000.0,0.0,15.0,27783.0,53.9,38.0,17.94,60 months,B,RENT,Source Verified,other,0.0,Current


In [154]:
# Getting summarized information
df.info()

numeric_features = ['loan_amnt', 'int_rate', 'annual_inc', 'inq_last_6mths', 
                   'open_acc', 'revol_bal', 'revol_util', 'total_acc', 'dti']

categorical_features = ['term', 'grade', 'home_ownership', 'verification_status',
                        'purpose', 'delinq_2yrs']


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   loan_amnt            10000 non-null  float64
 1   int_rate             10000 non-null  float64
 2   annual_inc           10000 non-null  float64
 3   inq_last_6mths       10000 non-null  float64
 4   open_acc             10000 non-null  float64
 5   revol_bal            10000 non-null  float64
 6   revol_util           9997 non-null   float64
 7   total_acc            10000 non-null  float64
 8   dti                  10000 non-null  float64
 9   term                 10000 non-null  object 
 10  grade                10000 non-null  object 
 11  home_ownership       10000 non-null  object 
 12  verification_status  10000 non-null  object 
 13  purpose              10000 non-null  object 
 14  delinq_2yrs          10000 non-null  float64
 15  loan_status          10000 non-null  

In [155]:
# =============================================================================
# Applying Transformation to numeric variable
# =============================================================================


# Imputer - missing Value
# StandardScaler
# MaxAbsScaler
# MinMaxScaler
# QuantileTransformers - Uniform Distribution
# PowerTransformer - Gaussian Distrbution
# Normalizer
# FunctionTransformer - Custom Transformer

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler


numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(missing_values=np.nan, strategy='median'))])


In [156]:
# =============================================================================
# Applying transformation to Categorical Variable
# =============================================================================


from sklearn.preprocessing import OneHotEncoder
   
categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
        ('onehot' , OneHotEncoder(sparse=False))])


In [157]:
# =============================================================================
# Creating Preprocessor
# =============================================================================

from sklearn.compose import ColumnTransformer
    
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])



In [158]:
# =============================================================================
# Dimensionality Reduction - PCA
# =============================================================================

from sklearn.decomposition import PCA

pca_model = PCA(n_components=3)


In [159]:
# =============================================================================
# Model with feature selection
# =============================================================================

from sklearn.linear_model import LogisticRegression
    
model = Pipeline(steps=[
        ('preprocess', preprocessor),
        ('scaling', StandardScaler()),
        ('feature_selection', pca_model ),
        ('lr_model', LogisticRegression())])


In [160]:
# =============================================================================
# Applying Model
# =============================================================================
X = df.drop('loan_status', axis=1)
X = X.iloc[1:1000, :]
y = np.where(df['loan_status'].isin(['Charged Off', 'Default']), 1, 0)
y = y[1:1000]

def file_modify(dataset):
    print("Start")
    dataset['home_ownership'] = np.where(
            dataset.home_ownership.isin(['NONE', 'ANY']), 'MORTGAGE', 
            dataset.home_ownership)
    dataset['purpose'] = np.where(
            dataset.purpose.isin(['debt_consolidation', 'credit_card', 'home_improvement']), 
            dataset.purpose, 
            'other')
    dataset['delinq_2yrs'] = np.where(
            dataset.delinq_2yrs > 0, 1, 0 )
    return dataset


In [161]:
X = file_modify(X)


Start


In [162]:
model.fit(X, y)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['loan_amnt', 'int_rate',
                                                   'annual_inc',
                                                   'inq_last_6mths', 'open_acc',
                                                   'revol_bal', 'revol_util',
                                                   'total_acc', 'dti']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                    

In [163]:
pred = model.predict(X)


In [164]:
# =============================================================================
# Dumping Model
# =============================================================================


import joblib

joblib.dump(model, 'model_with_feature_selection_3.joblib')


['model_with_feature_selection_3.joblib']